In [0]:
from __future__ import print_function
import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
from torchvision import datasets, transforms

In [0]:
class Net(nn.Module):
    def __init__(self):
        super(Net, self).__init__()                   #input -? OUtput? RF
        self.conv1 = nn.Conv2d(1, 8, 3, bias = False, padding = 1)
        self.conv2 = nn.Conv2d(8, 16, 3, bias = False, padding = 1) 
        self.conv9 = nn.Conv2d(16, 16, 3, bias = False, padding = 1) 
        self.pool1 = nn.MaxPool2d(2, 2)
        self.conv3 = nn.Conv2d(16, 8, 1, bias = False)
        self.conv4 = nn.Conv2d(8, 16, 3, bias = False, padding = 1) 
        self.conv5 = nn.Conv2d(16, 16, 3, bias = False, padding = 1) 
        self.pool2 = nn.MaxPool2d(2, 2)
        self.conv6 = nn.Conv2d(16, 8, 1, bias = False) 
        self.conv7 = nn.Conv2d(8, 16, 3, bias = False)
        self.conv8 = nn.Conv2d(16, 10, 3, bias = False)
        # self.conv9 = nn.Conv2d(8, 16, 3, bias = False)
        # self.conv10 = nn.Conv2d(16, 10, 3, bias = False)
        self.avgpool = nn.AdaptiveAvgPool2d(1)
        self.bn81 = nn.BatchNorm2d(8)
        self.bn82 = nn.BatchNorm2d(8)
        self.bn83 = nn.BatchNorm2d(8)
        self.bn161 = nn.BatchNorm2d(16)
        self.bn162 = nn.BatchNorm2d(16)
        self.bn163 = nn.BatchNorm2d(16)
        self.bn165 = nn.BatchNorm2d(16)
        self.bn321 = nn.BatchNorm2d(16)
        # self.bn165 = nn.BatchNorm2d(16)
        # self.bn84 = nn.BatchNorm2d(8)
        self.drop = nn.Dropout(0.05)

    def forward(self, x):
        x = self.drop(self.bn81(F.relu(self.conv1(x))))
        x = self.drop(self.bn161(F.relu(self.conv2(x))))
        x = self.drop(self.bn165(F.relu(self.conv9(x))))
        x = self.pool1(x)
        x = self.drop(self.bn82(F.relu(self.conv3(x))))
        x = self.drop(self.bn162(F.relu(self.conv4(x))))
        x = self.drop(self.bn321(F.relu(self.conv5(x))))
        x = self.pool2(x)
        x = self.drop(self.bn83(F.relu(self.conv6(x))))
        x = self.drop(self.bn163(F.relu(self.conv7(x))))
        
        # x = self.drop(self.bn84(F.relu(self.conv8(x))))
        # x = self.drop(self.bn165(F.relu(self.conv9(x))))
        x = self.conv8(x)
        x = self.avgpool(x)
        x = x.view(-1, 10)
        return F.log_softmax(x)

In [4]:
!pip install torchsummary
from torchsummary import summary
use_cuda = torch.cuda.is_available()
device = torch.device("cuda" if use_cuda else "cpu")
model = Net().to(device)
summary(model, input_size=(1, 28, 28))

----------------------------------------------------------------
        Layer (type)               Output Shape         Param #
            Conv2d-1            [-1, 8, 28, 28]              72
       BatchNorm2d-2            [-1, 8, 28, 28]              16
           Dropout-3            [-1, 8, 28, 28]               0
            Conv2d-4           [-1, 16, 28, 28]           1,152
       BatchNorm2d-5           [-1, 16, 28, 28]              32
           Dropout-6           [-1, 16, 28, 28]               0
            Conv2d-7           [-1, 16, 28, 28]           2,304
       BatchNorm2d-8           [-1, 16, 28, 28]              32
           Dropout-9           [-1, 16, 28, 28]               0
        MaxPool2d-10           [-1, 16, 14, 14]               0
           Conv2d-11            [-1, 8, 14, 14]             128
      BatchNorm2d-12            [-1, 8, 14, 14]              16
          Dropout-13            [-1, 8, 14, 14]               0
           Conv2d-14           [-1, 16,

/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:47: UserWarning: Implicit dimension choice for log_softmax has been deprecated. Change the call to include dim=X as an argument.


In [5]:


torch.manual_seed(1)
batch_size = 128

kwargs = {'num_workers': 1, 'pin_memory': True} if use_cuda else {}
train_loader = torch.utils.data.DataLoader(
    datasets.MNIST('../data', train=True, download=True,
                    transform=transforms.Compose([
                        transforms.ToTensor(),
                        transforms.Normalize((0.1307,), (0.3081,))
                    ])),
    batch_size=batch_size, shuffle=True, **kwargs)
test_loader = torch.utils.data.DataLoader(
    datasets.MNIST('../data', train=False, transform=transforms.Compose([
                        transforms.ToTensor(),
                        transforms.Normalize((0.1307,), (0.3081,))
                    ])),
    batch_size=batch_size, shuffle=True, **kwargs)



Extracting ../data/MNIST/raw/train-images-idx3-ubyte.gz to ../data/MNIST/raw



Extracting ../data/MNIST/raw/train-labels-idx1-ubyte.gz to ../data/MNIST/raw



Extracting ../data/MNIST/raw/t10k-images-idx3-ubyte.gz to ../data/MNIST/raw



Extracting ../data/MNIST/raw/t10k-labels-idx1-ubyte.gz to ../data/MNIST/raw
Processing...
Done!


In [0]:
from tqdm import tqdm

def train_eval(model, device, train_loader):
    model.eval()
    train_loss = 0
    correct = 0
    with torch.no_grad():
        for data, target in train_loader:
            data, target = data.to(device), target.to(device)
            output = model(data)
            train_loss += F.nll_loss(output, target, reduction='sum').item()  # sum up batch loss
            pred = output.argmax(dim=1, keepdim=True)  # get the index of the max log-probability
            correct += pred.eq(target.view_as(pred)).sum().item()

    train_loss /= len(train_loader.dataset)

    print('\n\nTrain set: Average loss: {:.4f}, Accuracy: {}/{} ({:.3f}%)\n'.format(
        train_loss, correct, len(train_loader.dataset),
        100. * correct / len(train_loader.dataset)))
  
def train(model, device, train_loader, optimizer, epoch):
    model.train()
    pbar = tqdm(train_loader)
    for batch_idx, (data, target) in enumerate(pbar):
        data, target = data.to(device), target.to(device)
        optimizer.zero_grad()
        output = model(data)
        loss = F.nll_loss(output, target)
        loss.backward()
        optimizer.step()
        pbar.set_description(desc= f'loss={loss.item()} batch_id={batch_idx}')
    train_eval(model, device, train_loader)


def test(model, device, test_loader):
    model.eval()
    test_loss = 0
    correct = 0
    with torch.no_grad():
        for data, target in test_loader:
            data, target = data.to(device), target.to(device)
            output = model(data)
            test_loss += F.nll_loss(output, target, reduction='sum').item()  # sum up batch loss
            pred = output.argmax(dim=1, keepdim=True)  # get the index of the max log-probability
            correct += pred.eq(target.view_as(pred)).sum().item()

    test_loss /= len(test_loader.dataset)

    print('\nTest set: Average loss: {:.4f}, Accuracy: {}/{} ({:.3f}%)\n'.format(
        test_loss, correct, len(test_loader.dataset),
        100. * correct / len(test_loader.dataset)))

In [7]:
epochs = 20
model = Net().to(device)
optimizer = optim.SGD(model.parameters(), lr=0.005, momentum=0.9)

for epoch in range(epochs):
    print("Epoch: {}/{}..\n ".format(epoch+1, epochs));
    train(model, device, train_loader, optimizer, epoch)
    test(model, device, test_loader)

  0%|          | 0/469 [00:00<?, ?it/s]

Epoch: 1/20..
 


/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:47: UserWarning: Implicit dimension choice for log_softmax has been deprecated. Change the call to include dim=X as an argument.
loss=0.16503207385540009 batch_id=468: 100%|██████████| 469/469 [00:13<00:00, 35.61it/s]




Train set: Average loss: 0.1034, Accuracy: 58318/60000 (97.197%)



  0%|          | 0/469 [00:00<?, ?it/s]

...



Train set: Average loss: 0.0157, Accuracy: 59716/60000 (99.527%)



  0%|          | 0/469 [00:00<?, ?it/s]


Test set: Average loss: 0.0203, Accuracy: 9925/10000 (99.250%)

Epoch: 20/20..
 


loss=0.0334927961230278 batch_id=468: 100%|██████████| 469/469 [00:13<00:00, 35.77it/s]




Train set: Average loss: 0.0172, Accuracy: 59684/60000 (99.473%)


Test set: Average loss: 0.0214, Accuracy: 9933/10000 (99.330%)

